In [1]:
!pip install accelerate peft bitsandbytes transformers trl

In [20]:
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer
from peft import AutoPeftModelForCausalLM, PeftModel
import os

In [3]:
model_id = "Qwen/Qwen2.5-0.5B"
dataset_id = "yahma/alpaca-cleaned"
trained_model_name = "qwen-instruct"

In [4]:
def get_model_and_tokenizer():
  tokenizer = AutoTokenizer.from_pretrained(model_id)
  tokenizer.pad_token = tokenizer.eos_token


  bnb_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_use_double_quant=True,
      bnb_4bit_quant_type="nf4",
      bnb_4bit_compute_dtype=torch.bfloat16
  )

  model = AutoModelForCausalLM.from_pretrained(
      model_id,
      device_map="auto",
      trust_remote_code=True,
      quantization_config=bnb_config
  )

  model.config.use_cache=False
  model.config.pretraining_tp=1

  return model, tokenizer


In [5]:
model, tokenizer = get_model_and_tokenizer()

In [6]:
def do_inference(model, tokenizer, input):
  input_tokens = tokenizer(input, return_tensors='pt').to("cuda")
  output_tokens = model.generate(**input_tokens, max_new_tokens=100)
  output = tokenizer.decode(output_tokens[0], skip_special_tokens=True)

  print_str = f"""
  -----------------
  Input :
  {input}

  Output :
  {output}
  -----------------
  """
  print(print_str)

In [34]:
do_inference(model, tokenizer, "Once Up on a time")
do_inference(model, tokenizer, "What are solar flares?")

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.



  -----------------
  Input :
  Once Up on a time

  Output :
  Once Up on a time, the world was a very different place. The world was much smaller, and the people were much more diverse. The world was much more peaceful, and the people were much more friendly. The world was much more peaceful, and the people were much more friendly. The world was much more peaceful, and the people were much more friendly. The world was much more peaceful, and the people were much more friendly. The world was much more peaceful, and the people were much more friendly. The world
  -----------------
  

  -----------------
  Input :
  What are solar flares?

  Output :
  What are solar flares? What causes them? What are their effects on the Earth? What are the effects of solar flares on the Earth? What are the effects of solar flares on the Earth? What are the effects of solar flares on the Earth? What are the effects of solar flares on the Earth? What are the effects of solar flares on the Earth? What 

## Data Preparation

In [8]:
def apply_chat_template(instruction, input, output)->str:
    return f"<|instruction|>\n{instruction}</s>\n<|user|>\n{input}</s>\n<|assistant|>\n{output}</s>"

In [9]:
dataset = load_dataset(dataset_id, split="train")

In [10]:
dataset

Dataset({
    features: ['output', 'input', 'instruction'],
    num_rows: 51760
})

In [11]:
def dataset_processor_func(examples):
  processed_prompts = [ apply_chat_template(instruction, input, output) for instruction, input, output in zip(examples["instruction"], examples["input"], examples["output"])]
  return {"text": processed_prompts}

In [12]:
dataset = dataset.map(dataset_processor_func, batched=True)

In [13]:
dataset

Dataset({
    features: ['output', 'input', 'instruction', 'text'],
    num_rows: 51760
})

In [14]:
dataset[0]["text"]

'<|instruction|>\nGive three tips for staying healthy.</s>\n<|user|>\n</s>\n<|assistant|>\n1. Eat a balanced and nutritious diet: Make sure your meals are inclusive of a variety of fruits and vegetables, lean protein, whole grains, and healthy fats. This helps to provide your body with the essential nutrients to function at its best and can help prevent chronic diseases.\n\n2. Engage in regular physical activity: Exercise is crucial for maintaining strong bones, muscles, and cardiovascular health. Aim for at least 150 minutes of moderate aerobic exercise or 75 minutes of vigorous exercise each week.\n\n3. Get enough sleep: Getting enough quality sleep is crucial for physical and mental well-being. It helps to regulate mood, improve cognitive function, and supports healthy growth and immune function. Aim for 7-9 hours of sleep each night.</s>'

## Training

In [15]:
peft_config = LoraConfig(
  r=8,
  lora_alpha=16,
  lora_dropout=0.05,
  bias="none",
  task_type="CAUSAL_LM"
)

In [16]:
training_arguments = TrainingArguments(
        output_dir=trained_model_name,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=16,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        warmup_ratio=0.05,
        save_strategy="epoch",
        logging_steps=10,
        num_train_epochs=3,
        max_steps=250,
        fp16=True,
        # push_to_hub=True
    )

In [17]:
trainer = SFTTrainer(
        model=model,
        train_dataset = dataset,
        # dataset_text_field = "text",
        peft_config=peft_config,
        args=training_arguments,
        tokenizer=tokenizer,
        # packing=False,
        # max_seq_length=1024
    )

<ipython-input-17-408652070c7c>:1: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Tokenizing train dataset:   0%|          | 0/51760 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/51760 [00:00<?, ? examples/s]

In [18]:
import torch
torch.cuda.empty_cache()

In [19]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: sujithappikatla (sujithappikatla-org) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss
10,1.958200
20,1.802500
30,1.632300
40,1.533900
50,1.454500
60,1.435700
70,1.403300
80,1.422600
90,1.364400
100,1.367200


TrainOutput(global_step=250, training_loss=1.4177499237060547, metrics={'train_runtime': 1045.4132, 'train_samples_per_second': 7.652, 'train_steps_per_second': 0.239, 'total_flos': 4272360904458240.0, 'train_loss': 1.4177499237060547})

In [24]:
def get_trained_model_and_tokenizer():
  bnb_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_use_double_quant=True,
      bnb_4bit_quant_type="nf4",
      bnb_4bit_compute_dtype=torch.bfloat16
  )

  model = AutoModelForCausalLM.from_pretrained(
      model_id,
      device_map="auto",
      trust_remote_code=True,
      quantization_config=bnb_config
  )

  peft_model = PeftModel.from_pretrained(model, trained_model_name+"/checkpoint-250", from_transformers=True, device_map="auto")

  model = peft_model.merge_and_unload()

  return model, tokenizer

In [25]:
instruct_model,tokenizer = get_trained_model_and_tokenizer()

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:355: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [26]:
def apply_generation_template(instruction, input)->str:
    return f"<|instruction|>\n{instruction}</s>\n<|user|>\n{input}</s>\n<|assistant|>\n"

In [33]:
prompt = apply_generation_template("what is solar flares?", "")
print(prompt)

do_inference(instruct_model, tokenizer, prompt)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


<|instruction|>
what is solar flares?</s>
<|user|>
</s>
<|assistant|>


  -----------------
  Input :
  <|instruction|>
what is solar flares?</s>
<|user|>
</s>
<|assistant|>


  Output :
  <|instruction|>
what is solar flares?</s>
<|user|>
</s>
<|assistant|>
Solar flares are large explosions in the Sun's atmosphere that can cause a large amount of energy to be released. These explosions are caused by the solar wind, which is a stream of charged particles that is emitted from the Sun. The energy from the solar wind is released through the formation of solar flares, which are large explosions that can cause a large amount of energy to be released. These explosions can be caused by a variety of factors, including solar wind storms, solar wind disturbances, and solar
  -----------------
  
